In [2]:
# Converts a dataBatch (iterable of [image_batch, label_batch]) 
# into a tuple of two lists: one list of image batches, another of corresponding label batches.
def from_DataBatch_to_list(dataBatch):
  val_dataset2 = []  # list to collect image batches
  val_dataset2_gt = []  # list to collect ground truth batches
  i = 0  # index counter for elements within batch (0: images, 1: labels)

  # Iterate over each batch in the provided dataBatch
  for batch in dataBatch:
    # Each batch is expected to be a sequence where index 0 is images and index 1 is labels
    for i in range(len(batch)):
      if i == 0:
        val_dataset2.append(batch[i])  # append image batch
      if i == 1:
        val_dataset2_gt.append(batch[i])  # append label batch
    
  # Return a tuple (images_list, labels_list)
  return (val_dataset2, val_dataset2_gt)


In [3]:
import torch

def topk_accuracy(model, k, validation_tuple, device=None):
    """
    Compute Top-K accuracy for a PyTorch model over a validation dataset.

    Args:
        model (torch.nn.Module): Trained PyTorch model.
        k (int): Number of top predictions to consider for accuracy.
        validation_tuple (tuple): A tuple (val_data, val_labels), where
            - val_data is a list of input batches (torch.Tensor) of shape [B, C, H, W]
            - val_labels is a list of corresponding label batches, either as class indices [B]
        device (torch.device, optional): Device to run inference on. If None, uses CUDA if available.

    return:
        float: Top-K accuracy over the entire validation set.
    """
    # Unpack validation data and labels
    val_data, val_labels = validation_tuple

    # Select device: use GPU if available and not specified
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Move model to device and set to evaluation mode
    model.to(device)
    model.eval()

    correct = 0  # Counter for correctly predicted samples
    total = 0    # Counter for total samples

    # Disable gradient computation for inference
    with torch.no_grad():
        # Iterate over batches of inputs and labels
        for x_batch, y_batch in zip(val_data, val_labels):
            # Move batch to the selected device
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)

            # Forward pass: obtain raw logits of shape [B, num_classes]
            outputs = model(x_batch)

            # Get the indices of the top-k predictions for each sample [B, k]
            topk_inds = outputs.topk(k, dim=1).indices

            # If labels are one-hot encoded, convert to class indices
            if y_batch.dim() > 1:
                y_true = y_batch.argmax(dim=1)
            else:
                y_true = y_batch  # Already class indices

            # Compare true labels against top-k predictions: [B, k] boolean tensor
            matches = topk_inds.eq(y_true.unsqueeze(1))

            # Count samples where the true label is among the top-k predictions
            correct += matches.any(dim=1).sum().item()
            total += x_batch.size(0)

    # Return the ratio of correct predictions
    return correct / total


In [4]:
def confusion_matrix(model, failure_types, validationTuple, threshold):
  precision = []
  recall = []

  val_dataset = validationTuple[0]
  val_dataset_gt = validationTuple[1]

  if (len(threshold)!=0):
    # Determine the predicted class for each data in the validation-set and build 
    # the corresponding list containing the ground truth
    for i in range(len(val_dataset)): 
      if i == 0:
        prediction = model.predict(val_dataset[i])
        prediction_gt = list(map(lambda x: np.asarray(x).argmax(), val_dataset_gt[i]))
        # Determine the predicted class
        predicted_class = list(map(lambda x: x[:8].argmax(), prediction))
        # Store the probability value for the predicted class
        predicted_class_probability = list(map(lambda x: x[:8].max(), prediction))

      prediction = model.predict(val_dataset[i])
      prediction_gt.extend(list(map(lambda x: np.asarray(x).argmax(), val_dataset_gt[i])))
      # Determine the predicted class
      predicted_class.extend(list(map(lambda x: x[:8].argmax(), prediction)))
      # Store the probability value for the predicted class
      predicted_class_probability.extend(list(map(lambda x: x[:8].max(), prediction)))

    # Calculate the confusion matrix for each threshold
    cm = []
    for l in range(len(threshold)):
      cm.append([[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
            [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
            [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
            [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
            [0,0,0,0,0,0,0,0,0]])

      for i in range(len(failure_types)):
        for j in range(len(prediction_gt)):
          if (prediction_gt[j] == i):
            # If the predicted value does not exceed the threshold, Classify the item as a negative class
            if (predicted_class_probability[j]<=threshold[l]):
                  cm[l][i][8] += 1
            else:
              for k in range(len(failure_types)):
                if (predicted_class[j] == k):
                  cm[l][i][k] += 1

    return cm

  else:
    # Determine the predicted class for each data in the validation-set and build 
    # the corresponding list containing the ground truth
    for i in range(len(val_dataset)):
      if i == 0:
        prediction = model.predict(val_dataset[i])
        prediction_gt = list(map(lambda x: np.asarray(x).argmax(), val_dataset_gt[i]))
        # Determine the predicted class
        predicted_class = list(map(lambda x: x.argmax(), prediction))

      prediction = model.predict(val_dataset[i])
      prediction_gt.extend(list(map(lambda x: np.asarray(x).argmax(), val_dataset_gt[i])))
      # Determine the predicted class
      predicted_class.extend(list(map(lambda x: x.argmax(), prediction)))

    # Calculate the confusion matrix for each threshold
    cm = [[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0]]

    for i in range(len(failure_types)):
      for j in range(len(prediction_gt)):
        if (prediction_gt[j] == i):
          for k in range(len(failure_types)):
            if (predicted_class[j] == k):
              cm[i][k] += 1

    return cm

In [5]:
def classes_proportion_correctly_classified(c_matrix, failure_types):
  dict_ = {}
  for i in range(len(failure_types)):
    dict_[failure_types[i]] = c_matrix[i][i]/np.sum(c_matrix[i])
  
  return dict_

In [6]:
def metrics_report(c_matrix):
  precision = 0
  recall = 0

  num = 0
  precision_den = 0
  recall_den = 0

  for i in range(len(c_matrix)):
  # Index 8 is assumed to represent the negative class
    if (i != 8):
      num += c_matrix[i][i]
      precision_den += c_matrix[i][i] + c_matrix[8][i]
      recall_den += c_matrix[i][i] + c_matrix[i][8]
    

  precision = num/precision_den
  recall = num/recall_den
  return (
      {'precision': precision, 
       'recall': recall,
       'f1-measure': 2*(precision*recall)/(precision+recall)})

In [7]:
def roc_auc_report(confusion_matrixes):
  tp_rates = []
  fp_rates = []

  for l in range(len(confusion_matrixes)):
    tp = 0; fn = 0
    fp = 0; tn = 0

    cm = confusion_matrixes[l]

    for i in range(len(cm)):
      if (i != 8):
        tp += cm[i][i] 
        fn += cm[i][8]
        fp += cm[8][i]
      
      if (i == 8):
        tn = cm[i][i]
  
    tp_rates.append(tp/(tp+fn))
    fp_rates.append(fp/(fp+tn))

  return (tp_rates, fp_rates)

In [8]:
import copy
import matplotlib.pyplot as plt

def roc_curve(tp_rates, fp_rates):
  tp = copy.deepcopy(tp_rates)
  #tp.extend([0])
  tp.reverse()

  fp = copy.deepcopy(fp_rates)
  #fp.extend([0])
  fp.reverse()

  plt.plot(fp, tp)
  plt.plot([0, 1], ls="--")
  plt.plot([0, 0], [1, 0] , c=".7"), plt.plot([1, 1] , c=".7")
  plt.xlabel("false positive rate") 
  plt.ylabel("true positive rate")
  plt.title("ROC curve")

In [10]:
import numpy as np

threshold1 = np.arange(0, 1, 0.01).tolist()
threshold2 = []